In [1]:
import os, sys
# module_path = os.path.abspath(os.path.join('../../Prompt-based-class-incremental-learning-in-Federated-Learning/CODA_Prompt_CVPR2023_Code/'))
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import torch, torchvision, numpy as np
from dataloaders import dataloader
from dataloaders.dataloader import iCIFAR100_Fed
import random

In [3]:
seed = 2023
np.random.seed(seed)
torch.manual_seed(seed)
random.seed(seed)
torch.cuda.manual_seed(seed)

In [3]:
from models import vision_transformer


In [4]:
model = vision_transformer.VisionTransformer(img_size=224, patch_size=16, embed_dim=768, depth=12,
                                           num_heads=12, use_grad_checkpointing=False, ckpt_layer=0,
                                           drop_path_rate=0
                                          )

In [5]:
import timm
load_dict = timm.models.vit_base_patch16_224(pretrained=True).state_dict()
del load_dict["head.weight"]
del load_dict["head.bias"]

In [6]:
model.load_state_dict(load_dict)

<All keys matched successfully>

In [7]:
model = torch.nn.Linear(768, 100)
model

Linear(in_features=768, out_features=100, bias=True)

In [8]:
model.state_dict()

OrderedDict([('weight',
              tensor([[ 0.0278, -0.0086,  0.0053,  ...,  0.0023, -0.0287, -0.0135],
                      [-0.0336, -0.0264,  0.0333,  ..., -0.0191,  0.0237, -0.0341],
                      [-0.0200, -0.0321, -0.0032,  ..., -0.0112, -0.0101, -0.0245],
                      ...,
                      [ 0.0112, -0.0077, -0.0156,  ...,  0.0037, -0.0022,  0.0004],
                      [ 0.0169,  0.0045, -0.0015,  ..., -0.0142, -0.0232, -0.0052],
                      [ 0.0154,  0.0060,  0.0152,  ...,  0.0307,  0.0008, -0.0304]])),
             ('bias',
              tensor([-0.0209,  0.0268,  0.0262, -0.0185, -0.0050,  0.0313, -0.0027, -0.0018,
                       0.0208, -0.0149,  0.0075,  0.0248, -0.0015,  0.0060, -0.0060,  0.0201,
                      -0.0202,  0.0045,  0.0237,  0.0330,  0.0091, -0.0041,  0.0115, -0.0254,
                      -0.0019,  0.0238, -0.0083, -0.0210,  0.0220, -0.0094, -0.0227,  0.0088,
                      -0.0287,  0.0092, -0.0

In [2]:
# !pip install fairscale
from models.vit_coda_p import vit_pt_imnet
import models
from models import  vit_coda_p


In [3]:
prompt_params = [10, [100, 8, 0 ,0 ,0]]

dual_prompt_params  = [10, [10, 20, 6]]

In [4]:
model_codap = vit_pt_imnet(out_dim=100, prompt_flag = 'codap', prompt_param=prompt_params)
# model_dual = vit_pt_imnet(out_dim=100, prompt_flag = 'dual', prompt_param=dual_prompt_params)
# model_l2p = vit_coda_p.vit_pt_imnet(out_dim=100, prompt_flag = 'l2p', prompt_param=prompt_params)
# model_base = vit_pt_imnet(out_dim=100, prompt_flag = "None", prompt_param=None)

++++++++++++++++++ in feature+++++++++++++++++++++
 freezing original model
freezing cls_token
freezing pos_embed
freezing patch_embed.proj.weight
freezing patch_embed.proj.bias
freezing blocks.0.norm1.weight
freezing blocks.0.norm1.bias
freezing blocks.0.attn.qkv.weight
freezing blocks.0.attn.qkv.bias
freezing blocks.0.attn.proj.weight
freezing blocks.0.attn.proj.bias
freezing blocks.0.norm2.weight
freezing blocks.0.norm2.bias
freezing blocks.0.mlp.fc1.weight
freezing blocks.0.mlp.fc1.bias
freezing blocks.0.mlp.fc2.weight
freezing blocks.0.mlp.fc2.bias
freezing blocks.1.norm1.weight
freezing blocks.1.norm1.bias
freezing blocks.1.attn.qkv.weight
freezing blocks.1.attn.qkv.bias
freezing blocks.1.attn.proj.weight
freezing blocks.1.attn.proj.bias
freezing blocks.1.norm2.weight
freezing blocks.1.norm2.bias
freezing blocks.1.mlp.fc1.weight
freezing blocks.1.mlp.fc1.bias
freezing blocks.1.mlp.fc2.weight
freezing blocks.1.mlp.fc2.bias
freezing blocks.2.norm1.weight
freezing blocks.2.norm1.bia

In [20]:
model_codap = torch.nn.DataParallel(model_codap)
model_codap.cuda()
for key, param in model_codap.named_parameters():
    if 'module.prompt.e_p_0' in key: 
        print(key, param)

module.module.prompt.e_p_0 Parameter containing:
tensor([[[ 2.3563e-02, -1.0315e-02, -1.0168e-02,  ..., -1.5726e-02,
          -2.1585e-02,  2.3653e-03],
         [-1.1744e-02, -1.0223e-02,  1.0154e-02,  ...,  8.8750e-03,
           1.4252e-02, -1.2140e-02],
         [ 3.5424e-03,  3.0761e-03,  6.4863e-03,  ...,  2.7715e-02,
           1.1071e-02, -7.9777e-03],
         ...,
         [ 1.2643e-02,  8.7863e-03,  1.7152e-02,  ...,  1.4557e-04,
           2.4014e-03, -2.2078e-03],
         [-2.2192e-02,  5.5119e-03,  6.7970e-03,  ...,  2.7172e-03,
           1.5036e-02,  4.2390e-03],
         [ 1.4510e-03, -8.8770e-03,  1.6154e-02,  ..., -2.0709e-02,
           1.1269e-02, -1.6108e-02]],

        [[-1.1156e-02, -7.2050e-04, -2.8015e-03,  ...,  2.0302e-02,
          -7.6398e-03, -2.2658e-02],
         [ 1.1487e-02, -7.3623e-04,  1.7722e-02,  ...,  9.5978e-03,
           2.4161e-02, -6.7583e-03],
         [ 1.8390e-02, -5.6575e-03,  1.1179e-02,  ...,  1.2168e-02,
           2.4071e-03,  1.1

In [19]:
model_codap.state_dict()['module.prompt.e_p_0']

tensor([[[ 2.3563e-02, -1.0315e-02, -1.0168e-02,  ..., -1.5726e-02,
          -2.1585e-02,  2.3653e-03],
         [-1.1744e-02, -1.0223e-02,  1.0154e-02,  ...,  8.8750e-03,
           1.4252e-02, -1.2140e-02],
         [ 3.5424e-03,  3.0761e-03,  6.4863e-03,  ...,  2.7715e-02,
           1.1071e-02, -7.9777e-03],
         ...,
         [ 1.2643e-02,  8.7863e-03,  1.7152e-02,  ...,  1.4557e-04,
           2.4014e-03, -2.2078e-03],
         [-2.2192e-02,  5.5119e-03,  6.7970e-03,  ...,  2.7172e-03,
           1.5036e-02,  4.2390e-03],
         [ 1.4510e-03, -8.8770e-03,  1.6154e-02,  ..., -2.0709e-02,
           1.1269e-02, -1.6108e-02]],

        [[-1.1156e-02, -7.2050e-04, -2.8015e-03,  ...,  2.0302e-02,
          -7.6398e-03, -2.2658e-02],
         [ 1.1487e-02, -7.3623e-04,  1.7722e-02,  ...,  9.5978e-03,
           2.4161e-02, -6.7583e-03],
         [ 1.8390e-02, -5.6575e-03,  1.1179e-02,  ...,  1.2168e-02,
           2.4071e-03,  1.1656e-02],
         ...,
         [-2.1148e-02,  1

In [13]:
for n, p in model_codap.named_parameters():
    if "feat" in n:
        p.requires_grad = False
    else: 
        print(n)
n_parameters = sum(p.numel() for p in model_codap.parameters() if p.requires_grad)
total_parameters = sum(p.numel() for p in model_codap.parameters())
print('number of params:', n_parameters)
print('non trainable parameters', total_parameters)
print("percent of train paramters ", n_parameters/ total_parameters * 100)

last.weight
last.bias
prompt.e_p_0
prompt.e_k_0
prompt.e_a_0
prompt.e_p_1
prompt.e_k_1
prompt.e_a_1
prompt.e_p_2
prompt.e_k_2
prompt.e_a_2
prompt.e_p_3
prompt.e_k_3
prompt.e_a_3
prompt.e_p_4
prompt.e_k_4
prompt.e_a_4
number of params: 3916900
non trainable parameters 89715556
percent of train paramters  4.365909519637821


In [14]:
for n, p in model_l2p.named_parameters():
    if "feat" in n:
        p.requires_grad = False
    else: 
        print(n)
n_parameters = sum(p.numel() for p in model_l2p.parameters() if p.requires_grad)
total_parameters = sum(p.numel() for p in model_l2p.parameters())
print('number of params:', n_parameters)
print('non trainable parameters', total_parameters)
print("percent of train paramters ", n_parameters/ total_parameters * 100)

last.weight
last.bias
prompt.freq_curr_0
prompt.freq_past_0
prompt.e_p_0
prompt.e_k_0
number of params: 768100
non trainable parameters 86566956
percent of train paramters  0.8872900648141075


In [15]:
for n, p in model_dual.named_parameters():
    if "feat" in n:
        p.requires_grad = False
    else: 
        print(n)
n_parameters = sum(p.numel() for p in model_dual.parameters() if p.requires_grad)
total_parameters = sum(p.numel() for p in model_dual.parameters())
print('number of params:', n_parameters)
print('non trainable parameters', total_parameters)
print("percent of train paramters ", n_parameters/ total_parameters * 100)


last.weight
last.bias
prompt.freq_curr_2
prompt.freq_past_2
prompt.freq_curr_3
prompt.freq_past_3
prompt.freq_curr_4
prompt.freq_past_4
prompt.g_p_0
prompt.g_p_1
prompt.e_p_2
prompt.e_k_2
prompt.e_p_3
prompt.e_k_3
prompt.e_p_4
prompt.e_k_4
number of params: 569956
non trainable parameters 86368672
percent of train paramters  0.6599105749825585


In [16]:
for n, p in model_base.named_parameters():
    if "feat" in n:
        p.requires_grad = False
    else: 
        print(n)
n_parameters = sum(p.numel() for p in model_base.parameters() if p.requires_grad)
total_parameters = sum(p.numel() for p in model_base.parameters())
print('number of params:', n_parameters)
print('non trainable parameters', total_parameters)
print("percent of train paramters ", n_parameters/ total_parameters * 100)


last.weight
last.bias
number of params: 76900
non trainable parameters 85875556
percent of train paramters  0.0895481829544137


In [17]:
model_codap.prompt.e_k_0.shape

torch.Size([100, 768])

In [33]:
model_dual.prompt.e_p_2.shape


torch.Size([20, 768])

In [20]:
#torch ones image
import torch
img1 = torch.randn(32, 3, 224, 224)
img2 = torch.randn(32, 3, 224, 224)


In [10]:
st1 = torch.load("/data/gaurav/Projects/promptFCL/_outputs/ICCV/CIFAR100/10-task/fedMoon1/vit/coda-iid_fedMoon_1/models/repeat-1/task-1/class.pth")
st2 = torch.load("/data/gaurav/Projects/promptFCL/_outputs/ICCV/CIFAR100/10-task/fedMoon1/vit/coda-iid_fedMoon_1/models/repeat-1/task-3/class.pth")


In [11]:
import copy
model1 = copy.deepcopy(model_codap)
model2 = copy.deepcopy(model_codap)
model1 = torch.nn.DataParallel(model1)
model2 = torch.nn.DataParallel(model2)
# model1.cuda(), model2.cuda()
model1.cuda()
model2.cuda()

model1.load_state_dict(st1)
model2.load_state_dict(st2)

<All keys matched successfully>

In [14]:
# model1.forward(img1.cuda(), train = True)

In [15]:
# nega = cos(logits1, logits2)
# nega

In [16]:
# posi = cos(logits1, logits3)
# posi

In [21]:
# pre logits layer
# logits1[2]

_, _, logits1 = model1(img1.cuda(), train  = True)
_, _, logits3 = model2(img1.cuda(), train  = True)
_, _, logits2 = model2(img2.cuda(), train  = True)

In [14]:
cos = torch.nn.CosineSimilarity(dim=1)

In [15]:
tau = 0.1

In [22]:
posi = cos(logits1, logits3)
nega = cos(logits1, logits2)
numerator = torch.exp(posi/tau)
denominator = torch.exp(posi/tau) + torch.exp(nega/tau)
fedmoonLoss = -torch.log(numerator/denominator)

In [23]:
fedmoonLoss

tensor([0.3754, 0.5885, 0.4507, 0.5819, 0.5680, 0.5168, 0.5410, 0.5828, 0.5709,
        0.6081, 0.5433, 0.5738, 0.5530, 0.5750, 0.4804, 0.5005, 0.5351, 0.5394,
        0.5487, 0.1577, 0.5285, 0.3295, 0.5620, 0.5999, 0.5823, 0.4637, 0.5695,
        0.5008, 0.4366, 0.3534, 0.5314, 0.6124], device='cuda:0',
       grad_fn=<NegBackward>)

In [24]:
model_codap = torch.nn.DataParallel(model_codap)
model_codap = model_codap.cuda()

In [25]:
import copy
m1 = copy.deepcopy(model_codap) #.load_state_dict(st1)
m2 = copy.deepcopy(model_codap) #.load_state_dict(st2)

In [34]:
m1.load_state_dict(st1)
m2.load_state_dict(st2)

<All keys matched successfully>

In [107]:
for i in range(100):
     a= model2.state_dict()['module.prompt.e_p_0'][i] - model1.state_dict()['module.prompt.e_p_0'][i]
     print( i , torch.sum(a))
     

0 tensor(0., device='cuda:0')
1 tensor(0., device='cuda:0')
2 tensor(0., device='cuda:0')
3 tensor(0., device='cuda:0')
4 tensor(0., device='cuda:0')
5 tensor(0., device='cuda:0')
6 tensor(0., device='cuda:0')
7 tensor(0., device='cuda:0')
8 tensor(0., device='cuda:0')
9 tensor(0., device='cuda:0')
10 tensor(-0.7681, device='cuda:0')
11 tensor(0.6289, device='cuda:0')
12 tensor(1.1336, device='cuda:0')
13 tensor(1.1710, device='cuda:0')
14 tensor(0.7873, device='cuda:0')
15 tensor(-1.1305, device='cuda:0')
16 tensor(-1.2147, device='cuda:0')
17 tensor(1.3906, device='cuda:0')
18 tensor(-0.8674, device='cuda:0')
19 tensor(-1.1766, device='cuda:0')
20 tensor(1.8422, device='cuda:0')
21 tensor(1.5355, device='cuda:0')
22 tensor(-1.3895, device='cuda:0')
23 tensor(-0.9359, device='cuda:0')
24 tensor(-1.2834, device='cuda:0')
25 tensor(1.8066, device='cuda:0')
26 tensor(2.0858, device='cuda:0')
27 tensor(-1.5423, device='cuda:0')
28 tensor(-0.6096, device='cuda:0')
29 tensor(-1.3317, device

In [106]:
# model2.state_dict()['module.prompt.e_p_0']